In [4]:
import cupy as np          
import datetime                      
import copy
import pandas as pd
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import cartopy.io.shapereader as shpreader
import matplotlib as mpl
import matplotlib.pyplot as plt
import salem
%run /mnt/d/work/sunhh/script/function/lambert_ticks.py
%run /mnt/d/work/sunhh/script/function/calculate.py
%run /mnt/d/work/sunhh/script/function/评分.py

In [5]:
#读取观测并处理
def YMD_todatetime(ds):
  # 读取年月日数据，转换为Timestape，由于本质上还是遍历所有行，因此这个步骤最费时间
    import pandas as pd
    from datetime import datetime
    time = datetime(  # datetime 只接收整形参数，返回一个datetime类型的日期
        ds['Year'].astype(int), ds['Mon'].astype(int), ds['Day'].astype(int), ds['Hour'].astype(int)
    )
    return pd.to_datetime(time)
def PreProcess(df_t):
    df_t.insert( # 插入日期列，此时并不以此为索引
        1, 'time',df_t.iloc[:, 1:5].apply(YMD_todatetime, axis=1)
    ) 
    df_t.drop(columns=['Year', 'Mon', 'Day', 'Hour'], inplace=True, errors='raise')
    return df_t


In [6]:
StaDir = '/mnt/d/work/sunhh/data/station/' # 文件路径，自定义
usecols = ['Station_Id_C', 'Year', 'Mon', 'Day', 'Hour', 'TEM', 'PRE_1h'] # 需要的变量
# na_values = [32700, 32744, 32766]  # 分别代表 微量、空白、缺测，读取时替换为Nan
df = pd.DataFrame() 
df_t = pd.read_csv(
    StaDir+'SURF_CHN_MUL_HOR_20220707-20220810.txt',
    # sep='\s+',  parse_dates=False, na_values=na_values,
    sep='\s+',  parse_dates=False,
    engine='python', usecols=usecols, encoding='utf-8')
df_t = PreProcess(df_t)  # 中间处理

data=df_t
station=pd.read_excel('../../data/station/SURF_CHN_LIST.xlsx')
data_1=pd.merge(data[['Station_Id_C','time','PRE_1h','TEM']],station,on='Station_Id_C')
data_2=data_1.loc[(data_1.time>='2022-07-25T01')&(data_1.time<='2022-07-27T00')]

In [7]:
data_2

,Station_Id_C,time,PRE_1h,TEM,Station_Name,Country,Province,City,Cnty,Station_Id_d,Lat,Lon,Alti
561,50136,2022-07-25 01:00:00,0.0,21.1,漠河,中国,黑龙江省,大兴安岭地区,漠河县,50136,52.9744,122.5108,438.5
562,50136,2022-07-25 02:00:00,0.0,24.6,漠河,中国,黑龙江省,大兴安岭地区,漠河县,50136,52.9744,122.5108,438.5
563,50136,2022-07-25 03:00:00,0.0,26.9,漠河,中国,黑龙江省,大兴安岭地区,漠河县,50136,52.9744,122.5108,438.5
564,50136,2022-07-25 04:00:00,0.0,27.7,漠河,中国,黑龙江省,大兴安岭地区,漠河县,50136,52.9744,122.5108,438.5
565,50136,2022-07-25 05:00:00,0.0,28.5,漠河,中国,黑龙江省,大兴安岭地区,漠河县,50136,52.9744,122.5108,438.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104862,59951,2022-07-26 20:00:00,0.0,26.2,万宁,中国,海南省,省直辖县级行政区划,万宁市,59951,18.8006,110.3269,40.4
2104863,59951,2022-07-26 21:00:00,0.0,26.4,万宁,中国,海南省,省直辖县级行政区划,万宁市,59951,18.8006,110.3269,40.4
2104864,59951,2022-07-26 22:00:00,0.0,26.5,万宁,中国,海南省,省直辖县级行政区划,万宁市,59951,18.8006,110.3269,40.4
2104865,59951,2022-07-26 23:00:00,0.0,26.3,万宁,中国,海南省,省直辖县级行政区划,万宁市,59951,18.8006,110.3269,40.4
